In [37]:
from enum import Enum

from langchain_community.document_loaders import PolarsDataFrameLoader
from langchain.chat_models import init_chat_model
import polars as pl
from pydantic import BaseModel, Field
from thefuzz import process

from mfg_capabilities.config import config
from mfg_capabilities.utils import get_sqlalchemy_engine


CATALOG = "manufacturing_dev"
SCHEMA = "work_agent_barney"


engine = get_sqlalchemy_engine(
    catalog=CATALOG,
    schema=SCHEMA
)


pl.Config.set_tbl_rows(20)  # to set the number of rows displayed)
# pl.Config.set_tbl_cols(10) # to set the number of columns
# pl.Config.set_fmt_str_lengths(50) # to set the max string length displayed

polars.config.Config

## Load line and product data

In [2]:
# I manually cleaned the Excel sheets from central planning
# and stored in a single excel file
PRODUCTS_PATH = config.data_dir / "from_central_planning" / "products_from_capacity.xlsx"

# Read all columns as strings first to handle concatenated CSVs with extra headers
products_raw = pl.read_excel(
    PRODUCTS_PATH,
    sheet_name="Sheet1",
    has_header=True,
    infer_schema_length=0  # Treat all columns as strings
)

# display(products_raw)

In [3]:
products_cleaned = (
    products_raw
    .filter(~pl.col("Product").str.contains("(?i)none"))
    .select("Plant Number", "Plant Name", "Line Name", "Product")
    .unique()
    .sort("Plant Number", "Plant Name", "Line Name", "Product")
)

display(products_cleaned)

Plant Number,Plant Name,Line Name,Product
str,str,str,str
"""049""","""State Avenue Foods""","""670 SPICE""","""K00106 HT ONION POWDER 4OZ."""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48302 PS POPPY SEED"""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48307 PS GROUND CLOVES"""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48309 PS WHOLE CLOVES"""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48463 PS GROUND CUMIN"""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48464 PS THYME LEAVES"""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48466 PS GROUND CINNAMON"""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48468 PS GROUND RED PEPPER"""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48470 PS GROUND GINGER"""


## Infer RCK line numbers

In [4]:
rck_products = (
    products_cleaned
    .filter(pl.col("Plant Number") == "714")
)

display(rck_products)

Plant Number,Plant Name,Line Name,Product
str,str,str,str
"""714""","""RCK Foods""","""Deli Modern""","""Buffalo Chicken Dip 10 oz."""
"""714""","""RCK Foods""","""Deli Modern""","""Candied Jalapeno Bacon Dip 12 …"
"""714""","""RCK Foods""","""Deli Modern""","""Margherita Pizza Dip 12 oz."""
"""714""","""RCK Foods""","""Deli Modern""","""Sweet and Sour Dressing"""
"""714""","""RCK Foods""","""Deli Multivac 1""","""Albacore Tuna Salad"""
"""714""","""RCK Foods""","""Deli Multivac 1""","""American Potato Salad"""
"""714""","""RCK Foods""","""Deli Multivac 1""","""Artisan Mac N Cheese"""
"""714""","""RCK Foods""","""Deli Multivac 1""","""BLT Pasta Salad"""
"""714""","""RCK Foods""","""Deli Multivac 1""","""Bistro Bow Tie Pasta Salad Bas…"


In [5]:
query = """
SELECT
    plant_num,
    plant_name,
    bsc_fp_num,
    line_desc_finished_product,
    dept_name_production_plant,
    dept_name_sales_plant,
    line_market_desc
FROM
    financials_item_info_silver
WHERE
    plant_num = '714'
"""
rck_bsc_items = (
    pl.read_database(query, engine)
    .select([
        pl.col("plant_num").str.strip_chars().alias("plant_num"),
        pl.col("plant_name").str.strip_chars().alias("plant_name"),
        pl.col("bsc_fp_num").str.strip_chars().alias("bsc_fp_num"),
        pl.col("line_desc_finished_product").str.strip_chars().alias("line_desc_finished_product"),
        pl.col("dept_name_production_plant").str.strip_chars().alias("dept_name_production_plant"),
        pl.col("dept_name_sales_plant").str.strip_chars().alias("dept_name_sales_plant"),
        pl.col("line_market_desc").str.strip_chars().alias("line_market_desc"),
    ])
)
display(rck_bsc_items)

[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.


plant_num,plant_name,bsc_fp_num,line_desc_finished_product,dept_name_production_plant,dept_name_sales_plant,line_market_desc
str,str,str,str,str,str,str
"""714""","""RCK Foods""","""D03189""","""6/3LB KRO SOTHRN POTATO SALAD""","""Deli MFG Salads""","""Deli MFG Salads""","""KRO SOTHRN POTATO SALAD"""
"""714""","""RCK Foods""","""D03218""","""12/1LB KRO SOUTHRN POTATO SLD""","""Deli MFG Salads""","""Deli MFG Salads""","""KROGER SOUTHERN POTATO SALAD"""
"""714""","""RCK Foods""","""D03219""","""12/1LB KRO MUSTRD POTATO SALAD""","""Deli MFG Salads""","""Deli MFG Salads""","""KROGER MUSTARD POTATO SALAD"""
"""714""","""RCK Foods""","""D03271""","""12/1LB KRO HMSTYL BAKED BEANS""","""Deli MFG Salads""","""Deli MFG Salads""","""KROGER HOMESTYLE BAKED BEANS"""
"""714""","""RCK Foods""","""D03322""","""6/3LB KRO MSTRD POTATO SALAD""","""Deli MFG Salads""","""Deli MFG Salads""","""KROGER MUSTARD POTATO SALAD"""
"""714""","""RCK Foods""","""D03373""","""6/3LB KRO HMSTYL BAKED BEANS""","""Deli MFG Salads""","""Deli MFG Salads""","""KROGER HOMESTYLE BAKED BEANS"""
"""714""","""RCK Foods""","""D03734""","""WHOLESOME@HOME CRANBERRY CELEB""","""Deli MFG Salads""","""Deli MFG Salads""","""WHOLESOME@HOME CRANBERRY CELEB"""
"""714""","""RCK Foods""","""D08314""","""2/5LBS KROGER HOMESTYLE COCKTA""","""Deli MFG Salads""","""Deli MFG Salads""","""KROGER HOMESTYLE COCKTAIL SAUC"""
"""714""","""RCK Foods""","""D21090""","""12/15OZ BFR MINI SUGAR CKY 36C""","""FROZEN MFT COOKIES""","""FROZEN MFT COOKIES""","""BFR MINI SUGAR CKY 36CT"""


In [48]:
# Get the list of choices from rck_bsc_items
choices = rck_bsc_items["line_market_desc"].to_list()

# Function to get the best match and score
def get_best_match_and_score(product_name):
    """Finds the best match and its score for a product name from a list of choices."""
    best_match = process.extractOne(product_name, choices, score_cutoff=0)
    if best_match:
        return {"match": best_match[0], "score": best_match[1]}
    else:
        return {"match": None, "score": None}

# Apply the function to create new columns with the best match and score
rck_products_with_match = (
    rck_products
    .with_columns(
        pl.col("Product")
        .map_elements(get_best_match_and_score, return_dtype=pl.Struct([pl.Field("match", pl.Utf8), pl.Field("score", pl.Int64)]))
        .alias("match_struct")
    )
    .with_columns([
        pl.col("match_struct").struct.field("match").alias("best_match_desc"),
        pl.col("match_struct").struct.field("score").alias("match_score"),
    ])
    .drop("match_struct")
)

rck_products_joined = (
    rck_products_with_match
    .join(
        rck_bsc_items.select("bsc_fp_num", "line_market_desc"),
        left_on="best_match_desc",
        right_on="line_market_desc",
        how="left"
    )
    .select(
        pl.col("Plant Number"),
        pl.col("Plant Name"),
        pl.col("Line Name"),
        pl.col("Product"),
        pl.lit(True).alias("Fuzzy Matched"),
        pl.col("best_match_desc").alias("Best Fuzzy Match"),
        pl.col("match_score").alias("Fuzzy Match Score"),
        pl.col("bsc_fp_num").alias("BSC Finished Product Number"),
        pl.concat_str(pl.col("bsc_fp_num"), pl.lit(" "), pl.col("Product")).alias("Updated Product"),
    )
)

display(rck_products_joined)

Plant Number,Plant Name,Line Name,Product,Fuzzy Matched,Best Fuzzy Match,Fuzzy Match Score,BSC Finished Product Number,Updated Product
str,str,str,str,bool,str,i64,str,str
"""714""","""RCK Foods""","""Deli Modern""","""Buffalo Chicken Dip 10 oz.""",true,"""CHICKEN MILANESE""",86,"""D65981""","""D65981 Buffalo Chicken Dip 10 …"
"""714""","""RCK Foods""","""Deli Modern""","""Candied Jalapeno Bacon Dip 12 …",true,"""PRVT SEL CAN JAL BCN DIP""",58,"""D95191""","""D95191 Candied Jalapeno Bacon …"
"""714""","""RCK Foods""","""Deli Modern""","""Margherita Pizza Dip 12 oz.""",true,"""PRVT SEL PZZA DP""",56,"""D95192""","""D95192 Margherita Pizza Dip 12…"
"""714""","""RCK Foods""","""Deli Modern""","""Sweet and Sour Dressing""",true,"""RNDY SWEET SOUR COLE SLAW""",59,"""D66163""","""D66163 Sweet and Sour Dressing"""
"""714""","""RCK Foods""","""Deli Multivac 1""","""Albacore Tuna Salad""",true,"""ALBACORE TUNA SALAD""",100,"""D66010""","""D66010 Albacore Tuna Salad"""
"""714""","""RCK Foods""","""Deli Multivac 1""","""American Potato Salad""",true,"""AMERICAN POTATO SALAD""",100,"""D66033""","""D66033 American Potato Salad"""
"""714""","""RCK Foods""","""Deli Multivac 1""","""Artisan Mac N Cheese""",true,"""RCK ARTISAN MAC N CHEESE""",95,"""D66151""","""D66151 Artisan Mac N Cheese"""
"""714""","""RCK Foods""","""Deli Multivac 1""","""BLT Pasta Salad""",true,"""2/5LB BGS RCK BLT PASTA SALAD""",90,"""D66155""","""D66155 BLT Pasta Salad"""
"""714""","""RCK Foods""","""Deli Multivac 1""","""Bistro Bow Tie Pasta Salad Bas…",true,"""BISTRO BOW TIE PASTA SA""",88,"""D66026""","""D66026 Bistro Bow Tie Pasta Sa…"


### Use LLM to vet the fuzzy matches

In [ ]:
matches_to_eval = (
    rck_products_joined
    .filter(pl.col("Fuzzy Match Score") < 90)  # Filter for low confidence matches
    .select(
        pl.col("Line Name").alias("Page Content"),
        "Line Name",
        "Product",
        pl.col("Best Fuzzy Match").alias("Matched Product"),
    )
)

loader = PolarsDataFrameLoader(matches_to_eval, page_content_column="Page Content")
docs_to_eval = loader.load()


class LikertConfidence(int, Enum):
    not_at_all_confident = 1
    slightly_confident = 2
    somewhat_confident = 3
    moderately_confident = 4
    quite_confident = 5
    very_confident = 6
    extremely_confident = 7


class ScoredMatch(BaseModel):
    """
    Represents a food production line, product, and that products best fuzzy match
    from another dataset.
    """
    line_name: str = Field("The production line on which the product is produced")
    product: str = Field("The product being produced on the line")
    best_fuzzy_match: str = Field("The best fuzzy match for the product from another dataset")
    likert_confidence: LikertConfidence = Field("Your confidence in the match being correct on a scale of 1 to 7, where 1 is not at all confident and 7 is extremely confident")
    reasoning: str = Field("Your reasoning for the match being correct or not")


class ScoredMatches(BaseModel):
    """
    Represents a list of scored matches for evaluation.
    """
    matches: list[ScoredMatch] = Field("A list of scored matches.")


prompt_template = """
You'll be given a list of food production lines, products, and their best fuzzy matches from another dataset.
Your task is to evaluate the matches and provide a confidence score for each one. Just having similar keywords
is not enough to be confident in the match. Make sure the products are of the same type, i.e. salad is not the
same thing as dresssing even if they are of a similar style. For each match, please provide your confidence in
the match being correct on a scale of 1 to 7, where 1 is not at all confident and 7 is extremely confident.
Please provide your confidence in the match being correct for each of the following matches:
{matches}
"""

llm = init_chat_model(model="gpt-4o", model_provider="openai")
structured_llm = llm.with_structured_output(ScoredMatches)
response = structured_llm.invoke(prompt_template.format(matches=docs_to_eval))

In [75]:
matches_llm_eval = (
    pl.DataFrame(response.matches)
    .select(
        pl.col("line_name").alias("Line Name"),
        pl.col("product").alias("Product"),
        pl.col("best_fuzzy_match").alias("Best Fuzzy Match"),
        pl.col("likert_confidence").alias("Likert Score"),
        pl.col("reasoning").alias("Reasoning"),
    )
)

display(matches_llm_eval)

Line Name,Product,Best Fuzzy Match,Likert Score,Reasoning
str,str,str,i64,str
"""Deli Modern""","""Buffalo Chicken Dip 10 oz.""","""CHICKEN MILANESE""",2,"""Buffalo chicken dip is a type …"
"""Deli Modern""","""Candied Jalapeno Bacon Dip 12 …","""PRVT SEL CAN JAL BCN DIP""",7,"""The matched product has very s…"
"""Deli Modern""","""Margherita Pizza Dip 12 oz.""","""PRVT SEL PZZA DP""",6,"""The key matching word here is …"
"""Deli Modern""","""Sweet and Sour Dressing""","""RNDY SWEET SOUR COLE SLAW""",3,"""Though both involve sweet and …"
"""Deli Multivac 1""","""Bistro Bow Tie Pasta Salad Bas…","""BISTRO BOW TIE PASTA SA""",7,"""This match is precise in wordi…"
"""Deli Multivac 1""","""Creamy Coleslaw Dressing""","""CREAMY COLE SLAW KIT""",5,"""The product line suggests rela…"
"""Deli Multivac 1""","""Presto Pasta Salad""","""KROGER SOUTHERN POTATO SALAD""",2,"""The key terms 'pasta' and 'pot…"
"""Deli Multivac 1""","""Raw Beet Red Cabbage Salad""","""TUNA SALAD SPREAD""",1,"""These two items are highly dif…"
"""Deli Multivac 1""","""Red Cabbage Cranberry""","""CRNBRY WALNUT RED CABBAGE""",6,"""Shared terms like 'Red Cabbage…"


In [78]:
updated_product_col = (
    pl.when(pl.col("Fuzzy Matched"))
    .then(pl.col("Updated Product"))
    .otherwise(pl.col("Product"))
)        

original_product_col = (
    pl.when(pl.col("Fuzzy Matched"))
    .then(pl.col("Product"))
    .alias("Original Product")
)

products_final = (
    products_cleaned
    .join(
        rck_products_joined,
        on=["Plant Number", "Plant Name", "Line Name", "Product"],
        how="left"
    )
    .join(
        matches_llm_eval,
        on=["Line Name", "Product", "Best Fuzzy Match"],
        how="left"
    )
    .with_columns(updated_product_col.alias("product"))
    .select(
        pl.col("Plant Number").alias("plant_num"),
        pl.col("Plant Name").alias("plant_name"),
        pl.col("Line Name").alias("line_name"),
        pl.col("product").str.extract(r"^[A-Za-z0-9]+", 0).alias("bsc_fp_num"),
        pl.col("product"),
        pl.col("Fuzzy Matched").alias("fuzzy_matched"),
        pl.col("Fuzzy Match Score").alias("fuzzy_match_score"),
        original_product_col.alias("original_product"),
        pl.col("Best Fuzzy Match").alias("best_fuzzy_match"),
        pl.col("Likert Score").alias("llm_confidence"),
        pl.col("Reasoning").alias("llm_reasoning")
    )
)

display(products_final)

plant_num,plant_name,line_name,bsc_fp_num,product,fuzzy_matched,fuzzy_match_score,original_product,best_fuzzy_match,llm_confidence,llm_reasoning
str,str,str,str,str,bool,i64,str,str,i64,str
"""049""","""State Avenue Foods""","""670 SPICE""","""K00106""","""K00106 HT ONION POWDER 4OZ.""",null,null,null,null,null,null
"""049""","""State Avenue Foods""","""670 SPICE""","""K48302""","""K48302 PS POPPY SEED""",null,null,null,null,null,null
"""049""","""State Avenue Foods""","""670 SPICE""","""K48307""","""K48307 PS GROUND CLOVES""",null,null,null,null,null,null
"""049""","""State Avenue Foods""","""670 SPICE""","""K48309""","""K48309 PS WHOLE CLOVES""",null,null,null,null,null,null
"""049""","""State Avenue Foods""","""670 SPICE""","""K48463""","""K48463 PS GROUND CUMIN""",null,null,null,null,null,null
"""049""","""State Avenue Foods""","""670 SPICE""","""K48464""","""K48464 PS THYME LEAVES""",null,null,null,null,null,null
"""049""","""State Avenue Foods""","""670 SPICE""","""K48466""","""K48466 PS GROUND CINNAMON""",null,null,null,null,null,null
"""049""","""State Avenue Foods""","""670 SPICE""","""K48468""","""K48468 PS GROUND RED PEPPER""",null,null,null,null,null,null
"""049""","""State Avenue Foods""","""670 SPICE""","""K48470""","""K48470 PS GROUND GINGER""",null,null,null,null,null,null


## Join with Formulations

In [24]:

query = "SELECT * FROM plm_spec_cross_references_silver"
plm_spec_cross_refs = pl.read_database(query, engine)

[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.


In [6]:
display(plm_spec_cross_refs)

spec_number,spec_type,spec_status,formula_management_formula,formula_management_formula_output,bsc_fp_num_consumer,bsc_fp_num_case,bsc_fp_num_pallet,bsc_raw_mat_num,bsc_num,spec_type_derived
str,str,str,str,str,str,str,str,str,str,str
"""5000072-004""","""Ingredient Specification""","""Deleted""",null,null,null,null,null,"""666666""","""666666""","""raw_material"""
"""5000072-005""","""Ingredient Specification""","""Draft""",null,null,null,null,null,"""786123""","""786123""","""raw_material"""
"""5000072-007""","""Ingredient Specification""","""Deleted""",null,null,null,null,null,"""300176""","""300176""","""raw_material"""
"""5000072-011""","""Ingredient Specification""","""Retired""",null,null,null,null,null,"""300176""","""300176""","""raw_material"""
"""5000072-012""","""Ingredient Specification""","""Draft""",null,null,null,null,null,"""300176""","""300176""","""raw_material"""
"""5000073-002""","""Ingredient Specification""","""Retired""",null,null,null,null,null,"""300182""","""300182""","""raw_material"""
"""5000074-001""","""Ingredient Specification""","""Retired""",null,null,null,null,null,"""300193""","""300193""","""raw_material"""
"""5000075-001""","""Ingredient Specification""","""Retired""",null,null,null,null,null,"""300202""","""300202""","""raw_material"""
"""5000076-001""","""Ingredient Specification""","""Deleted""",null,null,null,null,null,"""300248""","""300248""","""raw_material"""


In [21]:

query = """
SELECT
    output_material_spec_number,
    trade_spec_number,
    trade_spec_status,
    material_spec_status,
FROM
    plm_spec_map_trade_output_formulation_silver
WHERE
    array_contains(produced_by_plant_num, "049")
    OR array_contains(produced_by_plant_num, "102")
    OR array_contains(produced_by_plant_num, "714")
"""

plm_spec_map = pl.read_database(query, engine)

DatabaseError: (databricks.sql.exc.ServerOperationError) [TRAILING_COMMA_IN_SELECT] Trailing comma detected in SELECT clause. Remove the trailing comma before the FROM clause. SQLSTATE: 42601; line 7 pos 0
[SQL: 
SELECT
    output_material_spec_number,
    trade_spec_number,
    trade_spec_status,
    material_spec_status,
FROM
    plm_spec_map_trade_output_formulation_silver
WHERE
    array_contains(produced_by_plant_num, "049")
    OR array_contains(produced_by_plant_num, "102")
    OR array_contains(produced_by_plant_num, "714")
]
(Background on this error at: https://sqlalche.me/e/20/4xp6)

In [20]:
display(plm_spec_map)

trade_spec_number
str
"""5003986-001"""
"""5003745-001"""
"""5004051-001"""
"""5004052-001"""
"""5004056-001"""
"""5004062-001"""
"""5003986-001"""
"""5003745-001"""
"""5004051-001"""
